# Naive Bayes

Naive Bayes的优缺点
> 优点：在数据较少的情况下仍然有效，可以处理多类别问题。<br>
> 缺点：对输入数据的准备方式较为敏感。<br>
> 适用数据类型：标称型数据。<br>

朴素贝叶斯的一般过程
> 1. 收集数据：可以使用任何方法。本章使用RSS源。
> 2. 准备数据：需要数值型或者布尔型数据。
> 3. 分析数据：有大量特征时，绘制特征作用不大，此时使用直方图效果更好。
> 4. 训练算法：计算不同的独立特征的条件概率。
> 5. 测试算法：计算错误率。
> 6. 使用算法：文档分类等。

In [18]:
import numpy as np
import re
import random

## 1. 算法实现
### a.从文本中构建词向量

In [2]:
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]
    return postingList, classVec

def createVocabList(dataSet):
    """Return set of All words."""
    
    vocabSet = set()
    for document in dataSet:
        # 计算集合的并
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

def setOfWords2Vec(vocabList, inputSet):
    """
    遍历查看该单词是否出现，出现则将单词置1
    
    Parameters
    -----------
    vocabList : 所有单词的集合列表
    inputSet : 输入数据集
    
    Return
    -----------
    returnVec : 匹配列表
    """
    
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in inputSet:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

In [3]:
listOPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
myVocabList # 所有单词的集合，不含重复单词

['is',
 'food',
 'steak',
 'has',
 'so',
 'buying',
 'take',
 'him',
 'dalmation',
 'help',
 'garbage',
 'quit',
 'problems',
 'cute',
 'worthless',
 'mr',
 'to',
 'posting',
 'ate',
 'park',
 'my',
 'stupid',
 'flea',
 'maybe',
 'not',
 'love',
 'I',
 'licks',
 'stop',
 'how',
 'please',
 'dog']

In [4]:
# 查看输入样本中的单词在单词表中的分布情况
setOfWords2Vec(myVocabList, listOPosts[0])

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1]

### b. 从词向量计算概率

In [5]:
def trainNB0(trainMatrix, trainCategory):
    """朴素贝叶斯分类Demo
    
    Parameters
    -----------
    trainMatrix : 总的输入文本
    trainCategory : 文件对应的类别分类
    
    """
    numTrainDocs = len(trainMatrix)        # 训练文本的个数 
    numWords = len(trainMatrix[0])         # 训练样本向量，也是上面setOfWord2Vec得到的结果。
    # 因为侮辱性被标记为了1，所以计算数目时只要相加就可以了。这样也就得到了侮辱性文件个数的概率
    # 即trainCategory中1的个数
    pAbusive = np.sum(trainCategory) / float(numTrainDocs)
    # p0Num：非侮辱性样本中所有词汇出现的频度；p1Num：侮辱性样本中所有词汇出现的频度
    p0Num = np.zeros(numWords); p1Num = np.zeros(numWords)
    # p0Denom：非侮辱性样本中词汇总量；p1Denom：侮辱性样本中词汇总量
    p0Denom = 0.0; p1Denom = 0.0
    
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += np.sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += np.sum(trainMatrix[i])
    p1Vect = p1Num / p1Denom
    p0Vect = p0Num / p0Denom
    return p0Vect, p1Vect, pAbusive

In [6]:
# 测试Demo版分类函数
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
trainMat[:1]

[[0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1]]

In [7]:
p0V, p1V, pAb = trainNB0(trainMat, listClasses)
print(pAb, "\n")
print(p0V, "\n\n", p1V)

0.5 

[0.04166667 0.         0.04166667 0.04166667 0.04166667 0.
 0.         0.08333333 0.04166667 0.04166667 0.         0.
 0.04166667 0.04166667 0.         0.04166667 0.04166667 0.
 0.04166667 0.         0.125      0.         0.04166667 0.
 0.         0.04166667 0.04166667 0.04166667 0.04166667 0.04166667
 0.04166667 0.04166667] 

 [0.         0.05263158 0.         0.         0.         0.05263158
 0.05263158 0.05263158 0.         0.         0.05263158 0.05263158
 0.         0.         0.10526316 0.         0.05263158 0.05263158
 0.         0.05263158 0.         0.15789474 0.         0.05263158
 0.05263158 0.         0.         0.         0.05263158 0.
 0.         0.10526316]


### c. 根据实际情况修改分类器

在使用朴素贝叶斯分类器时：
> 1. 由于一些概率值可能会为0，这样最后的乘积也就为0，为了避免这种情况，可以使用拉普拉斯算子，将所有词汇的出现次数初始化为1，分母初始化为2。
> 2. 由于大部分概率值都很小，这样在做连积的时候会造成程序下溢。这里对成绩取对数，把求积转换成求和。

对上面代码进行修改，得到新的函数。

In [8]:
def trainNB(trainMatrix, trainCategory):
    """朴素贝叶斯分类Modify
    
    Parameters
    -----------
    trainMatrix : 总的输入文本
    trainCategory : 文件对应的类别分类
    
    """
    numTrainDocs = len(trainMatrix)        # 训练文本的个数 
    numWords = len(trainMatrix[0])         # 训练样本向量，也是上面setOfWord2Vec得到的结果。
    pAbusive = np.sum(trainCategory) / float(numTrainDocs)
    p0Num = np.ones(numWords); p1Num = np.ones(numWords)
    p0Denom = 2.0; p1Denom = 2.0
    
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += np.sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += np.sum(trainMatrix[i])
    p1Vect = np.log(p1Num / p1Denom)
    p0Vect = np.log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive

In [9]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = np.sum(vec2Classify * p1Vec) + np.log(pClass1)
    p0 = np.sum(vec2Classify * p0Vec) + np.log(1 - pClass1)   # 二分类，所以直接用1减
    if p1 > p0:
        return 1
    else:
        return 0
    
def testingNB():
    """测试朴素贝叶斯算法"""
    
    # 1. 加载数据集
    listOPosts, listClasses = loadDataSet()
    # 2. 创建单词集合
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    
    # 3. 计算单词是否出现并创建样本矩阵
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    
    # 4. 训练数据
    p0V, p1V, pAb = trainNB(np.array(trainMat), np.array(listClasses))
    # 5. 测试数据
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))
    
    testEntry = ['stupid', 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))

In [10]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


### d. 词袋模型
在前面的处理方法中，将每个词是否出现作为一个特征，这种被称为词集模型。如果一个词在文档中不止出现过一次，这可能意味着包含该词是否出现在文档中所不能表达的某种信息，这种被成为词袋模型。修改前面的setOfWords2Vec函数为bagOfWords2VecMN。

In [11]:
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

## 3. 使用朴素贝叶斯过滤垃圾邮件

使用朴素贝叶斯对电子邮件进行分类
> 1. 收集数据：提供文本文件。
> 2. 准备数据：将文本文件解析成词条向量。
> 3. 分析数据：检查词条确保解析的正确性。
> 4. 训练算法:使用我们之前建立的 trainNB0() 函数。
> 5. 测试算法:使用 classifyNB() ,并且构建一个新的测试函数来计算文档集的错误率。
> 6. 使用算法:构建一个完整的程序对一组文档进行分类,将错分的文档输出到屏幕上。

### a. 文本切分
对一个文本字符串可以使用string.split()方法进行切分。

In [12]:
mySent = 'This book is the best book on Python or M.L. I have ever laid eyes upon.'
mySent.split()

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M.L.',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon.']

可以看出文本被切分为单词，但是里面包含了标点符号。可以使用正则表达式。

In [17]:
regEx = re.compile('\\W+')       # W*会出现match empty patten
listOfTokens = regEx.split(mySent)
listOfTokens

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M',
 'L',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon',
 '']

In [16]:
# 去除其中的空字符并转换为小写
[tok.lower() for tok in listOfTokens if len(tok) > 0]

['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'm',
 'l',
 'i',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']

### b. 测试算法

In [32]:
def textParse(bigString):
    """文本切分
    将文本转换成长度大于2的单个词汇
    
    """
    import re
    listOfTokens = re.split(r'\W+', bigString)
    if len(listOfTokens) == 0:
        print(listOfTokens)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

def spamTest():
    docList = []; classList = []; fullText = []
    for i in range(1, 26):
        # 样本中大部分文件都是ASCII text，但有一些文件编码格式是windows 1252，需要异常处理。
        try:
            wordList = textParse(open('email/spam/{}.txt'.format(i)).read())
        except:
            wordList = textParse(open('email/spam/{}.txt'.format(i), encoding='Windows 1252').read())
        # append将可迭代元素整个加入，extend将可迭代元素逐个加入
        # docList为所有训练的样本
        docList.append(wordList)
        fullText.extend(wordList)   # fullText为所有词汇的集合
        classList.append(1)
        try:
            wordList = textParse(open('email/ham/{}.txt'.format(i)).read())
        except:
            wordList = textParse(open('email/ham/{}.txt'.format(i), encoding='Windows 1252').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    # 获取样本中出现的所有词汇
    vocabList = createVocabList(docList)
    # 设置训练集和测试集大小，测试集包含10个样本
    testSet = [int(num) for num in random.sample(range(50), 10)]
    trainingSet = list(set(range(50)) - set(testSet))
    
    # 训练模型
    trainMat = []; trainClasses = []
    for docIndex in trainingSet:
        # 生成包含所有样本的特征矩阵
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNB(np.array(trainMat), np.array(trainClasses))
    
    # 测试模型
    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    print('The error rate is: {}'.format(float(errorCount) / len(testSet)))

In [33]:
spamTest()

The error rate is: 0.1


## 4. 使用朴素贝叶斯从个人广告中获取区域倾向
使用朴素贝叶斯发现地域相关用词
> 1. 收集数据：从RSS源收集内容，需要对RSS源构建一个接口。
> 2. 准备数据：从文本文件解析成词条向量。
> 3. 分析数据：检查词条确保解析正确。
> 4. 训练算法：使用我们之前建立的trainNB0函数。
> 5. 测试算法：观察错误率，确保分类器可用。可以通过修改切分程序提升分类结果。
> 6. 使用算法：构建一个完整的程序，封装所有内容。给定两个RSS源，该程序会显示最常用的公共词。

In [6]:
import feedparser

In [ ]:
# 使用Craigslist上的个人广告，RSS源出现问题，不能连接
ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')

下面构建代码实现测试自动化

In [ ]:
def calcMostFreq(vocabList, fullText):
    """遍历词汇表中的每个词，并统计出现次数
       返回出现次数最高的100个单词。
    """
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token] = fullText.count(token)
    sortedFreq = sorted(freqDict.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedFreq[:30]

def localWords(feed1, feed0):
    """和上面的spamTest类似"""
    docList = []; classList = []; fullText = []
    minLen = min(len(feed1['entries']), len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    # 去掉高频词
    top30Words = calcMostFreq(vocabList, fullText)
    for pairW in top30Words:
        if pairW[0] in vocabList:
            vocabList.remove(pairW[0])
    
    # 获取训练数据和测试数据
    testSet = [int(num) for num in random.sample(range(2 * minLen), 20)]
    trainingSet = list(set(range(2 * minLen)) - set(testSet))
    
    trainMat = []; trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNB(np.array(trainMat), np.array(trainClasses))
    
    # 计算错误率
    errorCount = 0
    for docIndex in testSet:
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    print("The error rate is: {}".format(float(errorCount) / len(testSet)))
    return vocabList, p0V, p1V

In [ ]:
def testRss():
    ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')
    sf = feedparser.parse('http://sfbay.craigslist.org/stp/index.rss')
    vocabList, pSF, pNY = localWords(ny, sf)

In [ ]:
testRss()

显示地域相关的用词

In [ ]:
def getTopWords(ny, sf):
    import operator
    vocabList, p0V, p1V = locaWords(ny, sf)
    topNY = []; topSF=[]
    for i in range(len(p0V)):
        if p0V[i] > -6.0: topSF.append((vocabList[i], p0V[i]))
        if p1V[i] > -6.0: topNY.append((vocabList[i], p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair: pair[1], reverse=True)
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print("SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**")
    for item in sortedSF:
        print(item[0])
    print("NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**")
    for item in sortedNY:
        print(item[0])

## Sklearn实现

In [1]:
import numpy as np
from sklearn.naive_bayes import GaussianNB

In [2]:
# GaussianNB: 高斯朴素贝叶斯
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
Y = np.array([1, 1, 1, 2, 2, 2])
clf = GaussianNB()
clf.fit(X, Y)
print(clf.predict([[-0.8, -1]]))
clf_pf = GaussianNB()
clf_pf.partial_fit(X, Y, np.unique(Y))
print(clf_pf.predict([[-0.8, -1]]))

[1]
[1]


In [4]:
# MultinomialNB: 多项式朴素贝叶斯
X = np.random.randint(5, size=(6, 100))
y = np.array([1, 2, 3, 4, 5, 6])
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X, y)
print(clf.predict(X[2:3]))

[3]


In [5]:
# BernoulliNB: 伯努利朴素贝叶斯
X = np.random.randint(2, size=(6, 100))
Y = np.array([1, 2, 3, 4, 4, 5])
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(X, Y)
print(clf.predict(X[2:3]))

[3]
